In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib as mpl
PATH = '/Users/zachary/Desktop/'
zhfont = mpl.font_manager.FontProperties(fname=PATH + 'simhei.ttf')

import warnings
warnings.filterwarnings("ignore")


In [2]:
PATH = 'data/'
FQ_name = 'fq.xlsx'
FileName = 'data_Q3.csv'

freq_matrix = pd.read_excel(PATH + FQ_name).values[:,1:]

DF = pd.read_csv(PATH + FileName)
select_name = ['S-ZORB.AT-0006.DACA.PV',
 'S-ZORB.PT_2301.PV',
 'S-ZORB.PT_2101.PV',
 'S-ZORB.PT_2502.DACA',
 'S-ZORB.PT_1601.DACA',
 'S-ZORB.AT-0009.DACA.PV',
 'S-ZORB.AT-0005.DACA.PV',
 'S-ZORB.PT_9403.PV',
 'S-ZORB.PT_1602A.PV',
 'S-ZORB.PC_3101.DACA',
 'S-ZORB.PC_5101.PV',
 'S-ZORB.PT_2501.DACA',
 'S-ZORB.PC_1202.PV',
 'S-ZORB.PT_2905.DACA',
 'S-ZORB.PT_1201.PV',
 'S-ZORB.LC_1201.PV',
 'S-ZORB.PT_9001.PV']

var_data = DF[select_name].values
var_mean = np.mean(var_data, axis=0)
print(var_mean.shape)


C_idx, S_idx, RON_idx  = ['焦炭,wt%', '焦炭,wt%.1'], ['硫含量,μg/g', '硫含量,μg/g.1'], ['RON损失', '辛烷值RON']

C_data = DF[ C_idx].values
S_data = DF[S_idx].values
RON_data = DF[RON_idx].values

C_before = C_data[:,0]
C_after = C_data[:,1]

S_before = S_data[:,0]
S_after = S_data[:,1]

RON_before = RON_data[:,1]
RON_delta = RON_data[:,0]

import copy
signal_data = copy.deepcopy(var_data)

for j in range(signal_data.shape[1]):
    high_idx = np.where(signal_data[:,j] >= var_mean[j])[0]
    low_idx = np.where(signal_data[:,j] < var_mean[j])[0]
    
    signal_data[high_idx, j] = 1
    signal_data[low_idx, j] = -1


(17,)


In [3]:
def get_E_FCC(S_before, S_after, C_before, C_after, RON_delta, RON_before):
    S_DIFF = ((S_before - S_after)/S_before).reshape((-1,1))
    C_idx = ((1/(C_before+C_after))/2).reshape((-1,1))
    RON_DIFF = ((RON_delta)/RON_before).reshape((-1,1))
    
    E_FCC = (S_DIFF/(1+RON_DIFF)) ** C_idx 
    
    return E_FCC

E_FCC = get_E_FCC(S_before, S_after, C_before, C_after, RON_delta, RON_before)
E_FCC.shape

(325, 1)

In [4]:
def equal(matrix, y):
    sums = 0
    for i in range(len(matrix)):
        if y[i] == 0:
            pass
        else:
            if matrix[i] == y[i]:
                sums += 1
            else:
                sums -= 2
    return sums


RON_delta_low30 = RON_delta*0.7

change = []
change_scale = []
i = 0
for it in RON_delta_low30:
    idx = np.where((RON_delta <= it) & (S_after <=5))[0]
    new_data = signal_data[i,:]
    
    condt_data = signal_data[idx,:]
    
    sign_list = []
    for j in range(condt_data.shape[0]):
        sign = equal(new_data, condt_data[j,:])
        sign_list.append(sign)
    operate = []
    nums = []
    scale = []   
    if len(idx) == 0:
        print(i, '--------------------------------------')
        nums = [100 for tt in range(17)]
        scale = [100 for tt in range(17)]
    else:
        print('Number_{0} operation to achive goal:'.format(i+1))
        
        condt_idx = np.where(np.array(sign_list) == max(sign_list))[0]
        
        before_data = var_data[i,:]
        after_data = np.mean(var_data[condt_idx,:], axis=0) 
        
        for k in range(len(after_data)):
            if before_data[k] > after_data[k]:
                nums.append(-1)
                operate.append('down')
                scale.append(var_data[i,k] - var_mean[k])
            elif before_data[k] < after_data[k]:
                nums.append(1)
                operate.append('up')
                scale.append(var_mean[k] - var_data[i,k])
            else:
                scale.append(0)
                nums.append(0)
                operate.append('unchange')  
    
        for k in range(len(operate)):
            print('     Operate-{0}: {1}'.format(k, operate[k]))
        
    change.append(nums)
    change_scale.append(scale)
    i += 1

change = np.array(change)
change_scale = np.array(change_scale)

Number_1 operation to achive goal:
     Operate-0: down
     Operate-1: down
     Operate-2: down
     Operate-3: up
     Operate-4: up
     Operate-5: up
     Operate-6: down
     Operate-7: up
     Operate-8: down
     Operate-9: up
     Operate-10: down
     Operate-11: up
     Operate-12: down
     Operate-13: down
     Operate-14: up
     Operate-15: down
     Operate-16: down
Number_2 operation to achive goal:
     Operate-0: unchange
     Operate-1: unchange
     Operate-2: unchange
     Operate-3: unchange
     Operate-4: unchange
     Operate-5: unchange
     Operate-6: unchange
     Operate-7: unchange
     Operate-8: unchange
     Operate-9: unchange
     Operate-10: unchange
     Operate-11: unchange
     Operate-12: unchange
     Operate-13: unchange
     Operate-14: unchange
     Operate-15: unchange
     Operate-16: unchange
Number_3 operation to achive goal:
     Operate-0: down
     Operate-1: down
     Operate-2: down
     Operate-3: up
     Operate-4: down
     Opera

     Operate-0: up
     Operate-1: up
     Operate-2: up
     Operate-3: up
     Operate-4: up
     Operate-5: down
     Operate-6: down
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: up
     Operate-12: up
     Operate-13: up
     Operate-14: up
     Operate-15: up
     Operate-16: up
Number_98 operation to achive goal:
     Operate-0: up
     Operate-1: up
     Operate-2: up
     Operate-3: up
     Operate-4: up
     Operate-5: down
     Operate-6: up
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: down
     Operate-12: up
     Operate-13: up
     Operate-14: up
     Operate-15: up
     Operate-16: up
Number_99 operation to achive goal:
     Operate-0: up
     Operate-1: up
     Operate-2: up
     Operate-3: up
     Operate-4: up
     Operate-5: down
     Operate-6: up
     Operate-7: up
     Operate-8: up
     Operate-9: up
     Operate-10: down
     Operate-11: up
     Operate-12: u

Number_209 operation to achive goal:
     Operate-0: down
     Operate-1: up
     Operate-2: up
     Operate-3: down
     Operate-4: up
     Operate-5: down
     Operate-6: up
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: down
     Operate-12: up
     Operate-13: up
     Operate-14: up
     Operate-15: up
     Operate-16: up
Number_210 operation to achive goal:
     Operate-0: down
     Operate-1: up
     Operate-2: up
     Operate-3: down
     Operate-4: up
     Operate-5: down
     Operate-6: up
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: down
     Operate-12: up
     Operate-13: up
     Operate-14: up
     Operate-15: up
     Operate-16: up
Number_211 operation to achive goal:
     Operate-0: down
     Operate-1: up
     Operate-2: up
     Operate-3: down
     Operate-4: up
     Operate-5: down
     Operate-6: up
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Op

     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: down
     Operate-12: down
     Operate-13: up
     Operate-14: down
     Operate-15: down
     Operate-16: up
Number_233 operation to achive goal:
     Operate-0: down
     Operate-1: down
     Operate-2: down
     Operate-3: down
     Operate-4: down
     Operate-5: down
     Operate-6: down
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: down
     Operate-12: down
     Operate-13: up
     Operate-14: down
     Operate-15: down
     Operate-16: up
Number_234 operation to achive goal:
     Operate-0: down
     Operate-1: down
     Operate-2: down
     Operate-3: down
     Operate-4: down
     Operate-5: down
     Operate-6: down
     Operate-7: up
     Operate-8: up
     Operate-9: down
     Operate-10: down
     Operate-11: down
     Operate-12: down
     Operate-13: up
     Operate-14: down
     Operate-15: down
     Operate-16: up
Numbe

In [5]:
DF_standard = pd.read_excel(PATH + '附件四：354个操作变量信息.xlsx')
name_standard = DF_standard['位号'].values
scale_standard = DF_standard['取值范围'].values
delta = DF_standard['Δ值'].values

In [6]:
std_scale = []
for name in select_name:
    idx = np.where(name_standard == name)[0]
    std_scale.append(delta[idx][0])

In [7]:
std_scale = np.array(std_scale).reshape((-1,1))

In [8]:
change_scale = np.vstack((change_scale, std_scale.reshape((1,-1)) ))

print(change.shape, change_scale.shape)

(325, 17) (326, 17)


In [10]:
DF1 = pd.DataFrame(change)
DF1.columns = select_name
DF1.to_excel(PATH + '17个主操作变量变化趋势.xlsx')

DF2 = pd.DataFrame(change_scale)
DF2.columns = select_name
DF2.to_excel(PATH + '17个主操作变量变化数值.xlsx')